A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

# Implementation

In [1]:
%load_ext autoreload
%autoreload 2

The data structure has been implemented as a class.

In [85]:
class TSTNode:
    def __init__(self, char):
        self.char = char
        self.left = None
        self.middle = None
        self.right = None
        self.is_end_of_word = False

    def insert(self, word, index):
        if word[index] < self.char:
            if self.left is None:
                self.left = TSTNode(word[index])
            self.left.insert(word, index)
        elif word[index] > self.char:
            if self.right is None:
                self.right = TSTNode(word[index])
            self.right.insert(word, index)
        elif index < len(word) - 1:
            if self.middle is None:
                self.middle = TSTNode(word[index + 1])
            self.middle.insert(word, index + 1)
        else:
            self.is_end_of_word = True

    def search(self, word, index):
        if word[index] < self.char:
            if self.left is None:
                return False
            return self.left.search(word, index)
        elif word[index] > self.char:
            if self.right is None:
                return False
            return self.right.search(word, index)
        elif index < len(word) - 1:
            if self.middle is None:
                return False
            return self.middle.search(word, index + 1)
        else:
            return self.is_end_of_word

    def get_all_words(self, prefix, words):
        if self.is_end_of_word:
            words.append(prefix + self.char)
        if self.left is not None:
            self.left.get_all_words(prefix, words)
        if self.middle is not None:
            self.middle.get_all_words(prefix + self.char, words)
        if self.right is not None:
            self.right.get_all_words(prefix, words)

    def _to_string(self, indent='       '):
        repr_str = indent + repr(self)

        if self.left is not None:
            repr_str += '\n_lt_:' + self.left._to_string(indent + '  ')
        if self.middle is not None:
            repr_str += '\n_eq_:' + self.middle._to_string(indent + '  ')
        if self.right is not None:
            repr_str += '\n_gt_:' + self.right._to_string(indent + '  ')
        return repr_str

    def __repr__(self):
        return f'char: {self.char}, terminates: {self.is_end_of_word}'


class TernarySearchTree:
    def __init__(self):
        self._root = None
        self._terminates = False

    def insert(self, word):
        if word == "":
            self._terminates = True
        else:
            if self._root is None:
                self._root = TSTNode(word[0])
            self._root.insert(word, 0)

    def search(self, word):
        if self._root is None:
            return False
        return self._root.search(word, 0)

    def all_strings(self):
        if self._terminates:
            words = [""]
        else:
            words = []
        if self._root is not None:
            self._root.get_all_words("", words)
        return words

    def __len__(self):
        if self._root is None:
            return 0
        else:
            return len(self.all_strings())

    def __repr__(self):
        if self._root is None:
            return 'empty tree'
        else:
            return f'terminates: {self._terminates} \n' + self._root._to_string()

# Example usage

Create a new empty ternery search tree.

In [86]:
tst = TernarySearchTree()

Insert the string `'abc'` into the tree.

In [87]:
tst.insert('abc')

Display the tree.

In [88]:
print(tst)

terminates: False 
       char: a, terminates: False
_eq_:         char: b, terminates: False
_eq_:           char: c, terminates: True


Insert another string `'aqt'`.

In [89]:
tst.insert('aqt')

In [90]:
print(tst)

terminates: False 
       char: a, terminates: False
_eq_:         char: b, terminates: False
_eq_:           char: c, terminates: True
_gt_:           char: q, terminates: False
_eq_:             char: t, terminates: True


The tree should now contain two strings.

In [91]:
len(tst)

2

In [92]:
tst.all_strings()

['abc', 'aqt']

Search for the string `'ab'`, it should be found since it is a prefix of `'abc'`.

In [93]:
tst.search('ab')

False

The string `'ac'` should not be found.

In [94]:
tst.search('ac')

False

The tree can also contain the empty string.

In [95]:
tst.insert('')

In [96]:
len(tst)

3

In [97]:
print(tst)

terminates: True 
       char: a, terminates: False
_eq_:         char: b, terminates: False
_eq_:           char: c, terminates: True
_gt_:           char: q, terminates: False
_eq_:             char: t, terminates: True


In [98]:
tst.all_strings()

['', 'abc', 'aqt']

# Testing

The file `data/search_trees/insert_words.txt` contains words that we can insert into a tree.

In [99]:
tst = TernarySearchTree()
with open('data/search_trees/insert_words.txt') as file:
    words = [
        line.strip() for line in file
    ]
for word in words:
    tst.insert(word)
unique_words = set(words)

Verify the length of the data stucture.

In [100]:
assert len(tst) == len(unique_words), \
       f'{len(tst)} in tree, expected {len(unique_words)}'

Verify that all words that were inserted can be found.

In [101]:
for word in unique_words:
    assert tst.search(word), f'{word} not found'

Verify that all prefixes can be found.

In [102]:
for word in unique_words:
    for i in range(len(word) - 1, 0, -1):
        prefix = word[:i]
        assert tst.search(prefix), f'{prefix} not found'

AssertionError: combinatio not found

Chack that when searching for a exact match, only the inserted words are found, and no prefixes.

In [ ]:
for word in unique_words:
    for i in range(len(word), 0, -1):
        prefix = word[:i]
        if prefix not in unique_words:
            assert not tst.search(prefix, exact=True), \
                   f'{prefix} found'

Check that the empty string is in the tree (since it is a prefix of any string).

In [ ]:
assert tst.search(''), 'empty string not found'

Check that the empty string is not in the tree for an exact search.

In [ ]:
assert not tst.search('', exact=True), 'empty string found'

Check that words in the file `data/search_trees/not_insert_words.txt` can not be found in the tree.

In [ ]:
with open('data/search_trees/not_insert_words.txt') as file:
    for line in file:
        word = line.strip()
        assert not tst.search(word), f'{word} should not be found'

Check that all strings are returned.

In [ ]:
all_strings = tst.all_strings()
assert len(all_strings) == len(unique_words), \
       f'{len(all_strings)} words, expected {len(unique_words)}'
assert sorted(all_strings) == sorted(unique_words), 'words do not match'

If not output was generated, all tests have passed.